In [1]:
# Import required libraries
import csv
import sqlite3

import pandas as pd
from pycaret.regression import *

In [2]:
"""
Functions for predictions
"""


def get_data_from_db(curr):
    conn = sqlite3.connect(
        "predict.db", isolation_level=None, detect_types=sqlite3.PARSE_COLNAMES
    )

    sql = "SELECT * FROM " + curr + "_agg"
    db_df = pd.read_sql_query(sql, conn)
    db_df.rename(columns={"return_val": "actual"}, inplace=True)
    return db_df


def load_saved_models(currency_pairs):
    models = {}
    for currency in currency_pairs:
        models[currency] = load_model("./models/{}".format(currency))
    return models


def load_divider_values(currency_pairs):
    cutoff_values = {}
    for currency in currency_pairs:
        cutoff_values[currency] = {}

    with open("./regression_code/divider_list.csv", "r") as csvfile:
        csvreader = csv.reader(csvfile)
        fields = next(csvreader)

        for row in csvreader:
            try:
                cutoff_values[row[0]][row[1]] = [float(row[2]), float(row[3])]
            except:
                pass

    return cutoff_values


def substitute_cutoffs(df, cutoff_values, column):
    for index, row in df.iterrows():
        if row[column] < cutoff_values[column][0]:
            df.at[index, column] = 1
        elif cutoff_values[column][0] < row[column] < cutoff_values[column][1]:
            df.at[index, column] = 2
        else:
            df.at[index, column] = 3
    return df

In [22]:
def main(currency_pairs):
    # Load the saved models into a dictionary
    saved_models = load_saved_models(currency_pairs)

    # Load the cutoff values into a dictionary
    cutoff_divider_values = load_divider_values(currency_pairs)

    for currency in currency_pairs:
        df = get_data_from_db(currency)
        df = substitute_cutoffs(df, cutoff_divider_values[currency], "vol")
        df = substitute_cutoffs(df, cutoff_divider_values[currency], "fd")

        prediction = predict_model(saved_models[currency], df)
        prediction.Label /= 10000
        prediction.rename(columns={"Label": "predicted"}, inplace=True)

        prediction["error"] = (
            prediction["predicted"] - prediction["actual"]
        ) * 100 / prediction["actual"]

        prediction.to_csv(
            "./predictions/{}.csv".format(currency),
            columns=["actual", "predicted", "error"],
            index=False,
        )

In [23]:
# A dictionary defining the set of currency pairs we will be pulling data for
currency_pairs = [
    "EURUSD",
    "GBPUSD",
    "USDCAD",
    "USDCHF",
    "USDHKD",
    "USDAUD",
    "USDNZD",
    "USDSGD",
]

main(currency_pairs)

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
